# Emoji classification
## Imports
### Libraries

In [12]:
import os
import numpy as np
import pandas as pd 
from pathlib import Path
from PIL import Image
from skimage import io, color
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from warnings import filterwarnings
filterwarnings("ignore")

### Functions

In [13]:
def load_single_image(path):
    """
    This will be the function you use to preprocess your image.
    
    """
    img = Image.open(path).convert("RGBA")
    img = np.array(img)

    # RGBA -> RGB
    if img.ndim == 3 and img.shape[2] == 4:
        img = color.rgba2rgb(img) 
    # grayscale -> RGB
    elif img.ndim == 2:
        img = np.stack([img] * 3, axis=-1)

    # unint8 conversion
    if img.dtype != np.uint8:
        img = (img * 255).astype(np.uint8)

    # here you could add additional preprocessing

    return img.astype("float32") / 255.0


def imageLoader(files, labels, batch_size):
    """
    This will create a generator for learning by batches.
    """
    L = len(files)

    while True: 
        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)

            # Batch files (slices)
            batch_files = files[batch_start:limit]

            X_batch = []
            y_batch = []

            for f in batch_files:
                img_id = Path(f).stem       # nom sans extension
                img = load_single_image(f)
                X_batch.append(img)
                y_batch.append(labels[img_id])

            X = np.stack(X_batch)
            Y = np.array(y_batch)

            yield X, Y

            batch_start += batch_size
            batch_end += batch_size

# source code https://stackoverflow.com/questions/47200146/keras-load-images-batch-wise-for-large-dataset

### Dataset

In [14]:
PATH = Path("data")
if not PATH.exists():
    raise FileNotFoundError(f"Data folder not found at {PATH.resolve()}")

In [15]:
train_dir = PATH / "train"
if not train_dir.exists():
    raise FileNotFoundError(f"Train directory not found at {train_dir.resolve()}")
train_files = sorted([str(p) for p in train_dir.iterdir() if p.is_file()])

In [16]:
test_dir = PATH / "test"
if not test_dir.exists():
    raise FileNotFoundError(f"Test directory not found at {test_dir.resolve()}")
test_files = sorted([str(p) for p in test_dir.iterdir() if p.is_file()])
test_ids = [Path(f).stem for f in test_files]

In [17]:
labels_csv = PATH / "train_labels.csv"
if not labels_csv.exists():
    raise FileNotFoundError(f"Labels file not found at {labels_csv.resolve()}")

y_train_df = pd.read_csv(labels_csv)
y_train_dct = dict(zip(y_train_df["Id"], y_train_df["Label"]))

In [18]:
gen = imageLoader(
    files=train_files,
    labels=y_train_dct,
    batch_size=32,
)

## Training

In [19]:
unique_labels = y_train_df["Label"].unique().tolist()

In [20]:
def predict_with_my_amazing_model(test_files):
    prediction = np.random.choice(unique_labels, size= len(test_files))
    ids = np.arange(1,len(test_files)+1)
    prediction_as_df = pd.DataFrame()
    return prediction

y_test_pred = predict_with_my_amazing_model(test_files)
y_test_pred

array(['google', 'whatsapp', 'mozilla', ..., 'apple', 'whatsapp',
       'google'], shape=(9879,), dtype='<U9')

In [21]:
test_ids_sr = pd.Series(test_ids, name="Id")
y_test_pred_sr = pd.Series(y_test_pred, name="Label")
submission_df = pd.concat([test_ids_sr, y_test_pred_sr], axis=1)
submission_df.head()

,Id,Label
0,10001,google
1,10002,whatsapp
2,10003,mozilla
3,10004,apple
4,10005,facebook


In [22]:
submission_df.to_csv("submission.csv", index=False)